#### CSCE 470 :: Information Storage and Retrieval :: Texas A&M University :: Fall 2018


# Homework 2:  Hubs-and-Authorities + SEO

### 100 points [5% of your final grade]

### Due: October 4, 2018 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from a trust graph), implement and test Hubs and Authorities over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions (Google Classroom):* To submit your homework, rename this notebook as  `lastname_firstinitial_hw#.ipynb`. For example, my homework submission would be: `caverlee_j_hw2.ipynb`. Submit this notebook via **Google Classroom**. Your IPython notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so).

*Due:* October 4, 2018 by 11:59pm. For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after October 7 at 11:59pm.

# Part 1: Trust-based Hubs-and-Authorities (70 points)

## A Trust Graph

In this assignment, we're going to adapt the classic Hubs-and-Authorities approach to allow us to find not the most trustful users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Epinions users and their trust network (so user = node, trust another user = edge). Epinions was a review website where users could review products (like on Amazon) but also provide a "trust" link to other users. Over this trust graph, we can apply the Hubs-and-Authorities approach to order the users according to both their hub scores and authority scores. The main idea is that a user who is trusted by other users is more "trustful". So, a user with high trustHub score connects to lots of trusted users. A user with high authorityHub score is trusted by lots of good hubs.

Here is a toy example. Suppose you are given the following four relations of the form (UserID1 trust UserID2):

* teddiec trust jnbmoore
* matta75 trust tallgent
* nano trust dannec
* craftswoman trust ifif1938

So, "teddiec" is a node with a directed edge to the "jnbmoore" node. 

We provide you with a file containing these kinds of trust relationships: 

*epinions_network_trust.txt*.

You should build a graph of users and trust edges based on this file. Your data structure should support calculating Hubs-and-Authorities, so think carefully about how you want to approach this part.

In [4]:
#read the data
#import pandas as pd
#data = pd.read_csv('epinions_network_trust.txt', sep=' ', header = None, names =  ["UserID1", "Relation", "UserID2"])

graph = {}
with open('epinions_network_trust.txt') as fhook:
    for line in fhook:
        # Building the graph
        user1,relation,user2 = line.split()
        if user1 in graph: graph[user1]['out'][user2] = 1
        else: graph[user1] = {'out':{user2:1},'in':{},'hub':1,'auth':1}
        if user2 in graph: graph[user2]['in'][user1] = 1
        else: graph[user2] = {'out':{},'in':{user1:1},'hub':1,'auth':1}


In [5]:
def num_nodes(graph):
    return len(graph)

def num_edges(graph):
    return sum(len(graph[u]['in']) for u in graph)

In [6]:
# Call your function to print out the size of the graph, i.e., the number of nodes and edges
# How you maintain the graph is totally up to you
# However, if you encounter any memory issues, we recommend you write the graph into a file, and load it later.
# you may not use any existing graph libraries

print("Number of nodes:", num_nodes(graph))
print("Number of edges:", num_edges(graph))


Number of nodes: 1032
Number of edges: 1177


We will not check the correctness of your graph. However, this will affect the Hubs-and-Authorities results later.

## Hubs-and-Authorities Implementation

Your program will return the top-10 users with highest Hub scores and the top-10 users with the highest Authority scores.. The **output** should be like:

`TrustHub Scores`

`user1 - score1`

`user2 - score2`

`...`

`user10 - score10`


`TrustAuthority Scores`

`user1 - score1`

`user2 - score2`

` ...`

`user10 - score10`


You should follow these **rules**:

* If a user is never mentioned and does not mention anyone, their scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the calculation.
* There are implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions.
* Test your parsing and calculations using a handful of users, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your implementation? Describe it below:

Simply put, my implementation of the H-A calculation terminates once the number of iterations hits 1000.  I could have made one that stops once the precision hits a certain decimal point, but let's be real this graph is so small it's a joke. 500 gets the scores wayyy more precise than I need them to be but since the graph is miniscule it still runs instantly.  Let's get a real graph problem next time lol.

In [24]:
# Here add your code
# Insert as many cells as you want
from math import sqrt

# iters = iterations
def compute_ha(g, iters=500):
    for itr in range(iters):
        # Update Hubs
        sumsq = 1
        for u1 in g:
            hscore = sum(g[u2]['auth'] for u2 in g[u1]['out'])
            sumsq += hscore*hscore
            g[u1]['hub'] = hscore
        # Normalize Hubs
        sumsq = sqrt(sumsq)
        for u1 in g: g[u1]['hub'] /= sumsq

        # Update Authorities
        sumsq = 1
        for u1 in g:
            ascore = sum(g[u2]['hub'] for u2 in g[u1]['in'])
            sumsq += ascore*ascore
            g[u1]['auth'] = ascore
        # Normalize Authorities
        sumsq = sqrt(sumsq)
        for u1 in g: g[u1]['auth'] /= sumsq

In [25]:
def print_hubs(graph, top=20):
    print('TrustHub Scores')
    top_hub = sorted(((graph[u]['hub'], u) for u in graph), reverse=True)
    for i in range(top):
        print(top_hub[i][1], '-', top_hub[i][0])

def print_authorities(graph, top=20):
    print('TrustAuthority Scores')
    top_hub = sorted(((graph[u]['auth'], u) for u in graph), reverse=True)
    for i in range(top):
        print(top_hub[i][1], '-', top_hub[i][0])

In [26]:
# Now let's call your function on the graph you've built. Output the results.
compute_ha(graph)

print_hubs(graph, top=10)
print() # spacing
print_authorities(graph, top=10)

TrustHub Scores
scmrak - 0.5396812680927763
fuche_bu - 0.4161762103725233
hawgwyld - 0.24928333221357807
supergoldzero - 0.24637565540918674
brendametcalf - 0.20112741951420748
joecooper - 0.18124937142022568
ladyconsumer - 0.11672271392594415
dolphinboy - 0.11484786675780738
brendan2 - 0.10491411469288779
thewasp - 0.10319674566419838

TrustAuthority Scores
jankp - 0.5765344743266597
befus - 0.2723313220996898
vara - 0.24541751207680124
cripper - 0.23092239442058965
naphtalia - 0.2300516558200448
javelina - 0.22266870606315312
ed.williamson - 0.21599808150516858
bonniesayers - 0.2071596034248894
dierckx - 0.16966355866806968
pperky1954 - 0.1551706635472338


# Part 2: Search Engine Optimization (30 + 5 bonus points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **chkstong yibbibi** --- two terms, lower case, no quote. As of today (September 19, 2018), there are no hits for this query on either Google. Bing displays results using some very aggressive spell correction, but nothing that matches the query terms. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization). You'll find some good hints at [Moz's Beginner Guide to SEO](https://moz.com/beginners-guide-to-seo)


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you.
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

If you're stuck, take a look at the results for last year's SEO contest. The query last year was "qksame bapibko". You might find some inspiration.

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter what is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

http://www.nateleake.com/chkstong-yibbibi (top20 google),

but my home page http://www.nateleake.com/home (top10 bing) ranks higher on Bing
(Bing over-values home pages)

What's your strategy? (2-4 paragraphs)

Well I guess it's very multifaceted.  First part of my strategy was having my site up as early as possible. I think I dropped the chkstong page on my website within the first few minutes of the assignment appearing on Google Classrooms.  Immediately after that, I went and created a Reddit user (https://www.reddit.com/user/ChkstongYibbibi) and made/replied to some posts on the subject with other students from the class who were getting a head start.  After this, I went back to my website and wrote some details about the contest and other filler content (but none of it spam or meaningless since I know Google looks for and penalizes that kind of thing).  On my Chkstong page, I put up an offer to link to anyone's Chkstong page in return for them linking to mine, on the condition that their site wasn't spammy/gross (I didn't want to even remotely associate myself with spam sites).  I followed up on this with three different students, including the guy who ended up getting 1st place on Google.  After this I didn't really touch the site until the day the homework was due and the contest ended, at which point I removed content that was no longer relevant to the topic.  I also describe my strategy on-site in less detail than here.  Since I was kind of lazy I didn't bother putting up images or other media that might have helped my cause, but I did work on some basic link building from friends and other sites I controlled in a non-spammy way (example: "Oh, and I'm in an SEO contest for the keyword ChkstongYibbibi over on this site").  I didn't pay for anything other than the domain name I was already paying for (at ~5$ a year) and the time I put into the project.  I didn't bother to advertise at all so I only had ~20 organic clicks according to Google Search Console, so mostly I was relying on score from just having a well setup site without any nasty/spam.